#### Assignment #4
#### Name: Mingjie Zeng
#### ID: 671222265
#### Email: mzeng2@wpi.edu

Here is the link to the colab project:

https://colab.research.google.com/drive/1nHtuV_kXbc5GxluJqHISUWPGtBv1MTiw?usp=sharing

# **Naive Bayes Classifier**

## **Data pre-processing**

In [2]:
import pandas as pd

# read data
data_body = pd.read_csv("./emails/dbworld_bodies_stemmed.csv")
data_subject = pd.read_csv("./emails/dbworld_subjects_stemmed.csv")

print(data_body.shape)
print(data_body.head())
print(data_subject.shape)
print(data_subject.head())

(64, 3723)
   id  000euro  05102011  10th  11th  12noon  ...  özsu  û37  û42  û46  û56  CLASS
0   1        0         0     0     0       0  ...     0    0    0    0    0      0
1   2        0         0     0     0       0  ...     0    0    0    0    0      0
2   3        0         0     0     0       0  ...     0    0    0    0    0      0
3   4        0         0     0     0       0  ...     0    0    0    0    0      0
4   5        0         0     0     0       0  ...     0    0    0    0    0      0

[5 rows x 3723 columns]
(64, 231)
   id  10th  13th  1st  2nd  ...  wireless  workflow  workshop  zurich  CLASS
0   1     0     0    0    0  ...         0         0         0       0      0
1   2     0     0    0    0  ...         0         0         0       1      0
2   3     0     0    0    0  ...         0         0         0       0      0
3   4     0     0    0    0  ...         0         0         0       0      0
4   5     0     0    0    0  ...         0         0         0    

In [10]:
# delete the "id" column
data_body.drop(labels='id',axis=1,inplace = True)
data_subject.drop(labels='id',axis=1,inplace = True)

print(data_body.shape)
print(data_body.head())
print(data_subject.shape)
print(data_subject.head())

(64, 3722)
   000euro  05102011  10th  11th  12noon  12th  ...  özsu  û37  û42  û46  û56  CLASS
0        0         0     0     0       0     0  ...     0    0    0    0    0      0
1        0         0     0     0       0     0  ...     0    0    0    0    0      0
2        0         0     0     0       0     0  ...     0    0    0    0    0      0
3        0         0     0     0       0     0  ...     0    0    0    0    0      0
4        0         0     0     0       0     0  ...     0    0    0    0    0      0

[5 rows x 3722 columns]
(64, 230)
   10th  13th  1st  2nd  31st  ...  wireless  workflow  workshop  zurich  CLASS
0     0     0    0    0     0  ...         0         0         0       0      0
1     0     0    0    0     0  ...         0         0         0       1      0
2     0     0    0    0     0  ...         0         0         0       0      0
3     0     0    0    0     0  ...         0         0         0       0      0
4     0     0    0    0     0  ...         0

In [12]:
# train and test dataset
from sklearn.model_selection import train_test_split

# data_body
data_b = data_body.copy()
y_b = data_body['CLASS'].values
data_b.drop(['CLASS'], axis=1, inplace=True)

print(data_b.shape)
print(y_b)

# data_subject
data_s = data_subject.copy()
y_s = data_subject['CLASS'].values
data_s.drop(['CLASS'], axis=1, inplace=True)

print(data_s.shape)
print(y_s)

(64, 3721)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0]
(64, 229)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 0 0 0 0]


In [13]:
# split the dataset

# data_body
X_train_b, X_test_b, y_train_b, y_test_b = train_test_split(data_b, y_b, test_size=0.2, stratify=y_b)

# data_body
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(data_s, y_s, test_size=0.2, stratify=y_s)

print("Train data:", X_train_b.shape, y_train_b.shape)
print("Test data:", X_test_b.shape, y_test_b.shape)


Train data: (51, 3721) (51,)
Test data: (13, 3721) (13,)


In [14]:
print(type(X_train_b))
print(type(y_train_b))

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


## **a. You should implement from scratch a Naïve Bayes classifier (using the spam filter example discussed in class).Also implement Laplacian smoothing to handle words not in the dictionary. (60 points)**


In [112]:
import numpy as np
class NaiveBayesClassifier:
  def __init__(self):
    pass
  
  def fit(self,data,y):
    # turn data from pd to array
    X = np.array(data)
    X = X[:, 0:225]
    # merged array
    self.Xy = np.c_[X, y]
    # all class
    self.y_label = np.unique(self.Xy[:, -1])
    print("labels:", self.y_label)

  def predict(self,data_df):
    data = np.array(data_df)
    data = data[:, 0:225]
    # predicted label
    pred = []
    # predict every input email
    for i in range(len(data)):
      result = self.predict_one(data[i])
      pred.append(result)
    return np.array(pred)
  
  def predict_one(self, data_one):
    # prior probability
    prob = []
    # prior probability for each label
    for i in range(len(self.y_label)):
      # number of data with label i
      num_y_i = np.sum(self.Xy[:, -1] == self.y_label[i])
      # laplacian smoothing
      result = (num_y_i + 1) / (len(self.Xy) + len(self.y_label))
      prob.append(float(result))
    #print("prior probability:", prob)

    # conditional probability
    for i in range(len(data_one)):
      # word vector for every word in the email
      value = data_one[i]
      # conditional probability for each label
      for j in range(len(self.y_label)):
        # data with label j
        data_y_j = self.Xy[self.Xy[:,-1] == self.y_label[j]]
        # number of words with value i
        num_value_j = np.sum([data_y_j[:, i] == value])
        # laplacian smoothing
        result = (num_value_j + 1) / (len(data_y_j) + len(np.unique(self.Xy[:, i]))) 
        #result *= 10
        prob[j] *= float(result)
    print("conditional probability:", prob)
    
    return self.y_label[np.argmax(np.array(prob))]


## **b. Using the implemented algorithm, train and test the model for each dataset.**
**Use 80% of each class data to train your classifier and the remaining 20% to test it. Which dataset provides better classification i.e. email body or email subject? (30 points)**
**$$f - measure= \frac{2 \times Pre\times Rec}{ Pre + Rec}$$**
**where** 
**$$Pre=\frac{ TP}{TP + FP}$$; 
$$Rec= \frac{TP}{TP + FN}$$ ;
and TP is the number of true positives (class 1 members predicted as class 1), TN is the number of true negatives (class 2 members predicted as class 2),**
**FP is the number of false positives (class 2 members predicted as class 1),**
**and FN is the number of false negatives (class 1 members predicted as class 2).**

In [113]:
# data_body
clf_body = NaiveBayesClassifier()
clf_body.fit(X_train_b, y_train_b)
pred_body = clf_body.predict(X_test_b)

# data_subject
clf_subject = NaiveBayesClassifier()
clf_subject.fit(X_train_s, y_train_s)
pred_subject = clf_subject.predict(X_test_s)

labels: [0 1]
conditional probability: [2.8201819304000913e-19, 1.8708739834740405e-23]
conditional probability: [2.1137702445630414e-13, 4.02916757882641e-15]
conditional probability: [7.160103124256676e-10, 2.320800525404012e-12]
conditional probability: [5.89453914040532e-15, 6.515003271548286e-18]
conditional probability: [1.2888185623662022e-07, 1.3587232166910777e-10]
conditional probability: [8.215398438320387e-14, 1.21402260036953e-18]
conditional probability: [1.6419940164721106e-18, 2.5415745698525594e-19]
conditional probability: [6.789361268737836e-16, 1.8804962320009646e-17]
conditional probability: [5.298164130561774e-10, 3.771127348957554e-13]
conditional probability: [1.489527528653794e-29, 2.4568009067595123e-30]
conditional probability: [2.1831916647240826e-30, 2.285094821407966e-28]
conditional probability: [5.692042184215624e-27, 1.4404272034379522e-19]
conditional probability: [3.8117718239531125e-18, 9.936196878024423e-21]
labels: [0 1]
conditional probability: [1

In [114]:
# pred
print('pred_body:', pred_body)
print('pred_subject:', pred_subject)

pred_body: [0 0 0 0 0 0 0 0 0 0 1 1 0]
pred_subject: [1 1 0 0 0 0 1 0 1 0 1 1 1]


In [115]:
# show the result
from sklearn import metrics
from prettytable import PrettyTable

table = PrettyTable()

table.field_names = ['Dataset','Precision','Recall','F-score']
table.add_row(['email body',
               metrics.precision_score(y_test_b, pred_body),
               metrics.recall_score(y_test_b, pred_body),
               metrics.f1_score(y_test_b, pred_body)])
table.add_row(['email subject',
               metrics.precision_score(y_test_s, pred_subject),
               metrics.recall_score(y_test_s, pred_subject),
               metrics.f1_score(y_test_s, pred_subject)])
print(table)

+---------------+--------------------+---------------------+--------------------+
|    Dataset    |     Precision      |        Recall       |      F-score       |
+---------------+--------------------+---------------------+--------------------+
|   email body  |        0.5         | 0.16666666666666666 |        0.25        |
| email subject | 0.7142857142857143 |  0.8333333333333334 | 0.7692307692307692 |
+---------------+--------------------+---------------------+--------------------+


**From the results we can find that:**


**The email subject dataset can provide better classification with all three measuring scores much more higher.** 






## **c. Compare your classifier with the scikit-learn implementation(sklearn.naive_bayes.MultinomialNB). Repeat the analysis from (b). (30 points)**

In [116]:
from sklearn.naive_bayes import MultinomialNB

# data_body
clf_b = clf = MultinomialNB()
clf_b.fit(X_train_b, y_train_b)
pred_b = clf_b.predict(X_test_b)

# data_subject
clf_s = clf = MultinomialNB()
clf_s.fit(X_train_s, y_train_s)
pred_s = clf_s.predict(X_test_s)

In [117]:
# compare the result
table2 = PrettyTable()

table2.field_names = ['Classifier','Dataset','Precision','Recall','F-score']
table2.add_row(['Self-implemented','email body',
               metrics.precision_score(y_test_b, pred_body),
               metrics.recall_score(y_test_b, pred_body),
               metrics.f1_score(y_test_b, pred_body)])
table2.add_row(['Self-implemented','email subject',
               metrics.precision_score(y_test_s, pred_subject),
               metrics.recall_score(y_test_s, pred_subject),
               metrics.f1_score(y_test_s, pred_subject)])
table2.add_row(['Sklearn','email body',
               metrics.precision_score(y_test_b, pred_b),
               metrics.recall_score(y_test_b, pred_b),
               metrics.f1_score(y_test_b, pred_b)])
table2.add_row(['Sklearn','email subject',
               metrics.precision_score(y_test_s, pred_s),
               metrics.recall_score(y_test_s, pred_s),
               metrics.f1_score(y_test_s, pred_s)])
print(table2)

+------------------+---------------+--------------------+---------------------+--------------------+
|    Classifier    |    Dataset    |     Precision      |        Recall       |      F-score       |
+------------------+---------------+--------------------+---------------------+--------------------+
| Self-implemented |   email body  |        0.5         | 0.16666666666666666 |        0.25        |
| Self-implemented | email subject | 0.7142857142857143 |  0.8333333333333334 | 0.7692307692307692 |
|     Sklearn      |   email body  |        1.0         |  0.6666666666666666 |        0.8         |
|     Sklearn      | email subject | 0.7142857142857143 |  0.8333333333333334 | 0.7692307692307692 |
+------------------+---------------+--------------------+---------------------+--------------------+


**From the results we can find that:**


**For the self-implemented Naive Bayes classification:**

**The email subject dataset can provide better classification with all three measuring scores much more higher.** 

**For the sklearn Naive Bayes classification:**

**From the point of view of precision and f-measure measurement, the email body dataset can provide better classification.**  

**But from the point of view of recall measurement, the email subject dataset can provide better classification.**